Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [3]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Codigos-em-financas/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [4]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [5]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,14.06,69.89,0.627523,27.46,4.43,1.351371,52.32,272.12,3.16,1.60
BRSR6,11.03,1.30,9.949304,25.16,2.37,4.709832,36.63,232.09,4.68,NaN
SOMA3,5.95,80.87,21.677506,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
USIM3,6.24,2.17,2.176082,19.03,0.65,7.892759,16.68,167.31,9.63,0.82
JHSF3,4.46,25.39,15.523174,7.53,0.83,3.043392,11.86,165.92,5.37,2.55
BRAP3,18.00,9.00,1.186505,21.50,4.66,7.515149,47.48,163.78,3.88,-0.17
USIM5,6.42,2.17,91.471249,19.03,0.65,7.892759,16.68,159.81,9.93,0.82
MTRE3,3.63,23.11,1.960292,9.24,0.42,0.383963,9.34,157.30,8.72,4.45
LOGG3,23.00,48.82,8.746060,42.29,3.61,2.029546,58.61,154.83,6.39,3.79
BMGB4,4.04,10.97,2.874150,7.44,0.63,2.344594,10.27,154.21,6.35,NaN


outros criterios adicionais

In [6]:
'''
P/L > 0
'''
b_print(fundsSI[fundsSI['P/L'] > 0][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,14.06,69.89,0.627523,27.46,4.43,1.351371,52.32,272.12,3.16,1.60
BRSR6,11.03,1.30,9.949304,25.16,2.37,4.709832,36.63,232.09,4.68,NaN
SOMA3,5.95,80.87,21.677506,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
USIM3,6.24,2.17,2.176082,19.03,0.65,7.892759,16.68,167.31,9.63,0.82
JHSF3,4.46,25.39,15.523174,7.53,0.83,3.043392,11.86,165.92,5.37,2.55
BRAP3,18.00,9.00,1.186505,21.50,4.66,7.515149,47.48,163.78,3.88,-0.17
USIM5,6.42,2.17,91.471249,19.03,0.65,7.892759,16.68,159.81,9.93,0.82
MTRE3,3.63,23.11,1.960292,9.24,0.42,0.383963,9.34,157.30,8.72,4.45
LOGG3,23.00,48.82,8.746060,42.29,3.61,2.029546,58.61,154.83,6.39,3.79
BMGB4,4.04,10.97,2.874150,7.44,0.63,2.344594,10.27,154.21,6.35,NaN


In [7]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,14.06,69.89,0.627523,27.46,4.43,1.351371,52.32,272.12,3.16,1.60
BRSR6,11.03,1.30,9.949304,25.16,2.37,4.709832,36.63,232.09,4.68,0.00
SOMA3,5.95,80.87,21.677506,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
USIM3,6.24,2.17,2.176082,19.03,0.65,7.892759,16.68,167.31,9.63,0.82
JHSF3,4.46,25.39,15.523174,7.53,0.83,3.043392,11.86,165.92,5.37,2.55
BRAP3,18.00,9.00,1.186505,21.50,4.66,7.515149,47.48,163.78,3.88,-0.17
USIM5,6.42,2.17,91.471249,19.03,0.65,7.892759,16.68,159.81,9.93,0.82
BMGB4,4.04,10.97,2.874150,7.44,0.63,2.344594,10.27,154.21,6.35,0.00
BBAS3,26.06,20.38,390.653533,31.21,6.10,149.058993,65.45,151.15,4.26,0.00
SAPR3,4.96,10.90,0.809194,6.74,0.99,7.788753,12.25,146.98,5.03,2.09


remove empresas repetidas, mantem primeira ocorrencia

In [8]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
EUCA4,14.06,69.89,0.627523,27.46,4.43,1.351371,52.32,272.12,3.16,1.60
BRSR6,11.03,1.30,9.949304,25.16,2.37,4.709832,36.63,232.09,4.68,0.00
SOMA3,5.95,80.87,21.677506,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
USIM3,6.24,2.17,2.176082,19.03,0.65,7.892759,16.68,167.31,9.63,0.82
JHSF3,4.46,25.39,15.523174,7.53,0.83,3.043392,11.86,165.92,5.37,2.55
BRAP3,18.00,9.00,1.186505,21.50,4.66,7.515149,47.48,163.78,3.88,-0.17
MTRE3,3.63,23.11,1.960292,9.24,0.42,0.383963,9.34,157.30,8.72,4.45
LOGG3,23.00,48.82,8.746060,42.29,3.61,2.029546,58.61,154.83,6.39,3.79
BMGB4,4.04,10.97,2.874150,7.44,0.63,2.344594,10.27,154.21,6.35,0.00
BBAS3,26.06,20.38,390.653533,31.21,6.10,149.058993,65.45,151.15,4.26,0.00


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.